In [4]:
import os
import torch
from tqdm import tqdm
import numpy as np
import pickle, json
from tuned_lens import TunedLens

/root/miniconda3/envs/lens/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
from plot_utils import *
from get_dist import *

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_pth = './results/opt-6.7b/tuned_logits/'


l_data = []
label = []
for pkl_pth in os.listdir(file_pth):
    if pkl_pth[-4:] == '.pkl':
        with open(file_pth+pkl_pth, 'rb') as f:
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        for i in range(len(tuned_ans_list_r)):
            l_data.append(np.concatenate((tuned_ans_list_r[i][:,-1],logit_ans_list_r[i][:,-1])))
            # l_data.append(logit_ans_list_r[i][:,-1])
            label.append(1)
        for i in range(len(tuned_ans_list_z_o)):
            l_data.append(np.concatenate((tuned_ans_list_z_o[i][:,-1],logit_ans_list_z_o[i][:,-1])))
            # l_data.append(logit_ans_list_z_o[i][:,-1])
            label.append(0)
print(len(l_data))
X_train, X_test, y_train, y_test = train_test_split(l_data, label, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')  

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)